In [ ]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 5.1 MB/s eta 0:

In [ ]:
!pip install -U -q segmentation_models
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.5 MB/s eta 0:00:00


In [ ]:
!pip install -q efficientnet

In [ ]:
import gradio as gr
import PIL

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.models import load_model
import segmentation_models as sm
#import efficientnet.keras as efn
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2

Segmentation Models: using `tf.keras` framework.


In [ ]:
lr=1e-5

iou_score = [sm.metrics.IOUScore(threshold=0.5)]

def iou_loss(y_true, y_pred):
      y_true = tf.reshape(y_true, [-1])
      y_pred = tf.reshape(y_pred, [-1])
      intersection = tf.reduce_sum(tf.cast(y_true, tf.float32) * tf.cast(y_pred, tf.float32))
      score = (intersection + 1.) / (tf.reduce_sum(tf.cast(y_true, tf.float32)) +
      tf.reduce_sum(tf.cast(y_pred, tf.float32)) - intersection + 1.)
      return 1 - score

def mean_iou(y_true, y_pred):
      y_pred = tf.round(tf.cast(y_pred, tf.int32))
      intersect = tf.reduce_sum(tf.cast(y_true, tf.float32) * tf.cast(y_pred, tf.float32), axis=[1])
      union = tf.reduce_sum(tf.cast(y_true, tf.float32),axis=[1]) + tf.reduce_sum(tf.cast(y_pred, tf.float32),axis=[1])
      smooth = tf.ones(tf.shape(intersect))
      return tf.reduce_mean((intersect + smooth) / (union - intersect + smooth))

In [ ]:
# Load the segmentation model
model1 = load_model('/content/drive/MyDrive/UNET[Scratch].h5', compile=False)

model1.compile(optimizer = tf.keras.optimizers.Adam(lr),
              loss=iou_loss, metrics=[mean_iou],)

model2 = load_model('/content/drive/MyDrive/FPN.h5', compile=False,custom_objects = {'iou_score':iou_score})
model2.compile(optimizer = tf.keras.optimizers.Adam(lr),
              loss='categorical_crossentropy', metrics=[iou_score],)

In [ ]:
class_names = ['urban_land', 'agriculture_land', 'rangeland', 'forest_land', 'water','barren_land','unknown']

In [ ]:
def Unet(img):
  img_1=img.reshape(-1, 256, 256, 3)
  prediction=model1.predict(img_1).flatten()
  return {class_names[i]: float(prediction[i]) for i in range(7)}
iface1 = gr.Interface(fn=Unet, inputs = gr.inputs.Image(shape = (256, 256)),	outputs = gr.outputs.Label(num_top_classes=7), title="Unet")

<ipython-input-10-065d35eb8553>:5: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  iface1 = gr.Interface(fn=Unet, inputs = gr.inputs.Image(shape = (256, 256)),	outputs = gr.outputs.Label(num_top_classes=7), title="Unet")
<ipython-input-10-065d35eb8553>:5: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  iface1 = gr.Interface(fn=Unet, inputs = gr.inputs.Image(shape = (256, 256)),	outputs = gr.outputs.Label(num_top_classes=7), title="Unet")
<ipython-input-10-065d35eb8553>:5: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  iface1 = gr.Interface(fn=Unet, inputs = gr.inputs.Image(shape = (256, 256)),	outputs = gr.outputs.Label(num_top_classes=7), title="Unet")
<ipython-input-10-065d35eb8553>:5: GradioUnusedKwargWarning: You have unused

In [ ]:
def fpn(img):
  img_2=img.reshape(-1,256, 256, 3)
  prediction=model2.predict(img_2).flatten()
  return {class_names[i]: float(prediction[i]) for i in range(7)}
iface2 = gr.Interface(fn=fpn, inputs = gr.inputs.Image(shape = (256, 256)),	outputs = gr.outputs.Label(num_top_classes=7), title="FPN")

<ipython-input-11-0323f533f768>:5: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  iface2 = gr.Interface(fn=fpn, inputs = gr.inputs.Image(shape = (256, 256)),	outputs = gr.outputs.Label(num_top_classes=7), title="FPN")
<ipython-input-11-0323f533f768>:5: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  iface2 = gr.Interface(fn=fpn, inputs = gr.inputs.Image(shape = (256, 256)),	outputs = gr.outputs.Label(num_top_classes=7), title="FPN")
<ipython-input-11-0323f533f768>:5: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  iface2 = gr.Interface(fn=fpn, inputs = gr.inputs.Image(shape = (256, 256)),	outputs = gr.outputs.Label(num_top_classes=7), title="FPN")
<ipython-input-11-0323f533f768>:5: GradioUnusedKwargWarning: You have unused kwarg

In [ ]:
sample_images = [ ["201966_sat.jpg"], ["19849_sat.jpg"], ["191568_sat.jpg"], ["119896_sat.jpg"], ["115800_sat.jpg"]]

In [ ]:
# Combine both interfaces into a single Parallel interface
gr.Parallel(iface1, iface2, title="Land Segmentation: Unet",
            description="""Segmenting land from an image using a deep learning model.
            This application aims to provide a user-friendly interface for segmenting land areas in images.
            Firstly we get an intermediate output as a segmented image of the land cover, which is later converted into the percentage of the respective land classes.
            Overall, we aim to make land segmentation accessible to a wide range of users and facilitating further analysis and decision-making based on the segmented land regions.""",
            ).launch(share=True, debug=True, auth=("admin", "pass1234"))

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://4d8867cda226921748.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 5s 5s/step
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://4d8867cda226921748.gradio.live
